In [7]:
from ase import Atoms
from mace.calculators import mace_mp
from ase.build import surface
from ase.constraints import FixAtoms, UnitCellFilter
from ase.optimize import QuasiNewton
from ase.visualize import view
from ase.io import read, write
from ase.optimize import LBFGS
import numpy as np
import pandas as pd
from tabulate import tabulate
from cu2o_bulk import cu2o_bulk, cu2o111

bulk = cu2o_bulk()
bulk.calc = mace_mp(model="medium", dispersion=True, default_dtype="float64", device='cpu')
ucf = UnitCellFilter(bulk)
LBFGS(ucf).run(fmax=0.01)
E_bulk = bulk.get_potential_energy()
bulk_oxygens = bulk[bulk.symbols=='O']
print(f"Number of Oxygen atoms in bulk: {len(bulk_oxygens)}")
view(bulk)
#write('bulk.xyz', bulk)

#n_layers=3
#vacuum=10
#slab = cu2o111(bulk, n_layers, vacuum)
#view(slab)
'''

n_layers = { 3,4,5,6,7,8,9,10}
Cu2Otable={}

n_layers_list=[]
slab_oxygens_list=[]
E_surf_list=[]

for n_layers in range(3,11):
  vacuum=10
  slab = cu2o111(bulk, n_layers, vacuum)
  #write('slab.xyz', slab)
  slab_oxygens = slab[slab.symbols=='O']
  Cu2Otable[n_layers]=len(slab_oxygens)
  print(f"Number of Oxygen atoms in slab: {len(slab_oxygens)}")

  slab.calc = mace_mp(model="large", dispersion=True, default_dtype="float64", device='cuda')
  E_slab = slab.get_potential_energy()
  E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
  print(f'{n_layers=} {E_cleav=}')

  qn = LBFGS(slab, trajectory='Cu2O111.traj')
  #write('Cu2O111.traj', slab)
  qn.run(fmax=0.01)
  E_slab = slab.get_potential_energy()
  #t = read('Cu2O111.traj')
  #view(t)

  ##Calc Surface Energy
  E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
  print(f'{n_layers=} {E_surf=}')

  n_layers_list.append(n_layers)
  slab_oxygens_list.append(len(slab_oxygens))
  E_surf_list.append(E_surf)

df = pd.DataFrame({'Number of Layers': n_layers_list, 'Number of Oxygen Atoms':slab_oxygens_list, 'Surface Energy': E_surf_list})
table=tabulate(df, headers = 'keys', tablefmt = 'fancy_grid')
df.to_csv('Cu2O111.csv', index=False)
print(table)


#Fixing bottom layer, replicating bulk
#bottom_Cu_z = np.min(slab[slab.symbols=='Cu'].positions[:,2])
#mask1=slab.positions[:, 2] < bottom_Cu_z + 1.0
#slab.set_constraint(FixAtoms(mask=mask1))

'''

Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/5yyxdm76
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using TorchDFTD3Calculator for D3 dispersion corrections (see https://github.com/pfnet-research/torch-dftd)
       Step     Time          Energy         fmax
LBFGS:    0 13:33:06      -30.569296        0.3076
LBFGS:    1 13:33:06      -30.573217        0.2874
LBFGS:    2 13:33:07      -30.598950        0.0247
LBFGS:    3 13:33:07      -30.599045        0.0017
Number of Oxygen atoms in bulk: 2


'\n\nn_layers = { 3,4,5,6,7,8,9,10}\nCu2Otable={}\n\nn_layers_list=[]\nslab_oxygens_list=[]\nE_surf_list=[]\n\nfor n_layers in range(3,11):\n  vacuum=10\n  slab = cu2o111(bulk, n_layers, vacuum)\n  #write(\'slab.xyz\', slab)\n  slab_oxygens = slab[slab.symbols==\'O\']\n  Cu2Otable[n_layers]=len(slab_oxygens)\n  print(f"Number of Oxygen atoms in slab: {len(slab_oxygens)}")\n\n  slab.calc = mace_mp(model="large", dispersion=True, default_dtype="float64", device=\'cuda\')\n  E_slab = slab.get_potential_energy()\n  E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])\n  print(f\'{n_layers=} {E_cleav=}\')\n\n  qn = LBFGS(slab, trajectory=\'Cu2O111.traj\')\n  #write(\'Cu2O111.traj\', slab)\n  qn.run(fmax=0.01)\n  E_slab = slab.get_potential_energy()\n  #t = read(\'Cu2O111.traj\')\n  #view(t)\n\n  ##Calc Surface Energy\n  E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])\n  print(f\'{n_layers=} {E_surf=}\')\n\n  n_layers_list.append(n_layers